## IMPORT MODULES

In [ ]:
# update packages if necessary

# import pkg_resources
# from subprocess import call

# packages = [dist.project_name for dist in pkg_resources.working_set]
# call("pip install --upgrade " + ' '.join(packages), shell=True)

In [5]:
# !pip3 install webdriver-manager
!pip3 install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [13]:
# Modules for web scraping
# from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# URL encoding
from requests.utils import requote_uri
# Module for data manipulation
import pandas as pd
# Module for regular expression
import re

from dotenv import load_dotenv
load_dotenv()

# Module for file management
import os
# Module for timing
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import xlrd
from datetime import date
# from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager



### IMPORT EXCEL

In [ ]:
df = pd.read_excel('daftar_link_product_to_follow.xlsx',engine='openpyxl')


In [ ]:
# df['product_link'][]


**Geckodriver path**

In [2]:
# Options for Chrome windows
# options = webdriver.ChromeOptions()
options = webdriver.FirefoxOptions()
options.add_argument("--start-maximized")
# options.add_argument('window-size=2560,1440')
options.add_argument('disable-notifications')
options.add_argument('--disable-infobars')
# options.add_argument("--no-sandbox")
# options.add_argument('start-maximized')

# supaya gak muncul mozilla nya
# options.add_argument('--headless')

In [3]:
# DRIVER_PATH = 'chromedriver'
# driver = webdriver.Chrome(executable_path = DRIVER_PATH,options=options)
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)
driver.maximize_window()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current firefox version is 93.0
[WDM] - Get LATEST driver version for 93.0
[WDM] - Driver [/home/juliustn/.wdm/drivers/geckodriver/linux64/v0.30.0/geckodriver] found in cache
/tmp/ipykernel_68004/3609049643.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


In [4]:
main_url = "https://shopee.co.id/buyer/login?next=https%3A%2F%2Fshopee.co.id%2F"
driver.get(main_url)
WebDriverWait(driver, 10).until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR, '._3KEKx7')))
email_field = driver.find_element_by_name("loginKey")
email_field.send_keys(os.environ.get('PHONE'))
password_field = driver.find_element_by_name("password")
password_field.send_keys(os.environ.get('PASSWORD'))

password_field.send_keys(Keys.RETURN)

# login_btn = driver.find_element_by_xpath('//button[contains(text(), "Log in")]')
# if(login_btn.is_enabled()):
#     print(login_btn.is_enabled())
#     login_btn.click()

WebDriverWait(driver, 10).until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR, '._13vxdj')))

verification_btn = driver.find_element_by_xpath('//button')
if(verification_btn.is_enabled()):
    print(verification_btn.is_enabled())
    verification_btn.click()

# LOGIN MANUAL

/tmp/ipykernel_68004/3814809643.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email_field = driver.find_element_by_name("loginKey")
/tmp/ipykernel_68004/3814809643.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password_field = driver.find_element_by_name("password")
/tmp/ipykernel_68004/3814809643.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  verification_btn = driver.find_element_by_xpath('//button')


True


In [5]:
def scroll_window():
    scroll_height = driver.execute_script('return document.documentElement.scrollHeight;')
    for iter in range(0,scroll_height,200):
        driver.execute_script('window.scrollTo(0, {});'.format(iter))
        time.sleep(3)
    pass

In [6]:
user_list = []

In [7]:
def get_user_info(users):
    link = []
    for user in users:
        username = user.text
        username_link = user.get_attribute('href')
        user_list.append({
            'username':username,
            'user_link': username_link,
        })
        link.append(username_link)
    return link


In [8]:
def auto_follow_function(url):
    driver.get(url)

    WebDriverWait(driver, 30).until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR, '._193wCc')))
#     WebDriverWait(driver, 30).until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR, '._1VFgvr')))
    # WebDriverWait(driver, 30).until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR, '.product-ratings__list')))

    scroll_window()

    star_five_comment_btn = driver.find_element_by_xpath('//*[contains(text(), "5 bintang")]')

    star_five_comment_btn.click()

    for i in range(5000):
        comment_prev_btn = driver.find_element_by_css_selector('.shopee-icon-button--left')
        comment_next_btn = driver.find_element_by_css_selector('.shopee-icon-button--right')

        users = driver.find_elements_by_xpath('//div[@class="shopee-product-comment-list"]/div/div/a')

        ## TAMBAH USER KE USER_LIST
        link_user = get_user_info(users)

        link_user

        for user_href in link_user:
             #open new window with specific href
            driver.execute_script("window.open('" +user_href +"');")
            # switch to new window
            driver.switch_to.window(driver.window_handles[1])

            ### KLIK FOLLOW
            
            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[contains(text(), "ikuti")]'))).click()
            except:
                pass

            #close the new window
            driver.close()
            
            #back to main window
            driver.switch_to.window(driver.window_handles[0])

        # WebDriverWait(driver, 4)
        comment_next_btn.click()
        
        WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH,'//div[@class="shopee-product-comment-list"]')))



## PER PRODUK MANUAL

In [9]:
auto_follow_function("https://shopee.co.id/Correction-Tape-Pita-Koreksi-Joyko-CT-522-i.64925304.1068271033")


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54345): Max retries exceeded with url: /session/fac47d1e-bb43-4cde-a6d2-0f175c662801/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7faa2afeb3a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# add user list to dataframe and convert to excel
today = date.today()

df_user = pd.DataFrame.from_dict(user_list)


In [ ]:
df_user

In [ ]:
df_user=df_user.drop_duplicates('username')

In [ ]:
df_user.to_excel("user_follow_list_"+today.strftime("%b_%d_%Y")+".xlsx")